<a href="https://colab.research.google.com/github/kimdoyeomg/machine/blob/master/jdclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
maker=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, #회전
    zoom_range=0.15, #확대
    width_shift_range=0.2, #이동
    height_shift_range=0.2, #이동
    shear_range=0.15, # 굴절 
    horizontal_flip=True, # 가로반전
    fill_mode='nearest')

train_ds=maker.flow_from_directory(
    '/content/drive/MyDrive/dataset/',
    class_mode='categorical', # 2개면 바이너리 몇개더있으면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=32,
    subset='training',
    target_size=(150,150),
)

non_aug=ImageDataGenerator(rescale=1./255,validation_split=0.2)

valid_ds=non_aug.flow_from_directory(
    '/content/drive/MyDrive/dataset/',
    class_mode='categorical', # 2개면 바이너리 몇개더있으면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=32,
    subset='validation',
    target_size=(150,150),
)

Found 2614 images belonging to 8 classes.
Found 518 images belonging to 8 classes.


In [4]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

#가중치 파일
open('inception_v3.h5', 'wb').write(r.content)

87910968

In [5]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model=InceptionV3(input_shape=(150,150,3), include_top=False, weights=None)#마지막층 출력층 레이러 가져올지 말지 
inception_model.load_weights('inception_v3.h5')
inception_model.summary()

for i in inception_model.layers:
  i.trainalbe=False #가중치 고정

#fine tuning 예를들어 인셉션 모델을 mixed 6부터 트레이닝 조금 하고싶다면?
#mixded6부터 트레이닝 쭉할려면?
unfreeze=False
for i in inception_model.layers:
  if i.name=='mixed6':
    unfreeze=True
  if unfreeze ==True:
    i.trainable = True
lastlayer=inception_model.get_layer('mixed7')


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [6]:
import tensorflow as tf
layer1=tf.keras.layers.Flatten()(lastlayer.output)
layer2=tf.keras.layers.Dense(512,activation='relu')(layer1)
# layer3=tf.keras.layers.Dense(256,activation='relu')(layer2)
# layer4=tf.keras.layers.Dense(512,activation='relu')(layer3)
drop1=tf.keras.layers.Dropout(0.2)(layer2)
layer5=tf.keras.layers.Dense(8,activation='softmax')(drop1)

model=tf.keras.Model(inception_model.input,layer5)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=valid_ds, epochs=50)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/50
82/82 [==============================] - 1832s 22s/step - loss: 1.6183 - accuracy: 0.4021 - val_loss: 1.9541 - val_accuracy: 0.4846
Epoch 2/50
82/82 [==============================] - 28s 336ms/step - loss: 0.9409 - accuracy: 0.6897 - val_loss: 0.4088 - val_accuracy: 0.8707
Epoch 3/50
82/82 [==============================] - 27s 334ms/step - loss: 0.6210 - accuracy: 0.7972 - val_loss: 0.2864 - val_accuracy: 0.9228
Epoch 4/50
82/82 [==============================] - 27s 333ms/step - loss: 0.4794 - accuracy: 0.8412 - val_loss: 0.2555 - val_accuracy: 0.9189
Epoch 5/50
82/82 [==============================] - 28s 335ms/step - loss: 0.4164 - accuracy: 0.8669 - val_loss: 0.1563 - val_accuracy: 0.9537
Epoch 6/50
82/82 [==============================] - 28s 338ms/step - loss: 0.2967 - accuracy: 0.9074 - val_loss: 0.1097 - val_accuracy: 0.9691
Epoch 7/50
82/82 [==============================] - 27s 334ms/step - loss: 0.2523 - accuracy: 0.9220 - val_loss: 0.0942 - val_accuracy: 0.9788

In [7]:
model.save('/content/drive/MyDrive/jordan_ai_class')